# Training and Evaluation of GNNs and LLMs
In this notebook, we train the models on the [MovieLens Dataset](https://movielens.org/) after the Pytorch Geometrics Tutorial on [Link Prediction](https://colab.research.google.com/drive/1xpzn1Nvai1ygd_P5Yambc_oe4VBPK_ZT?usp=sharing#scrollTo=vit8xKCiXAue).

First we import all of our dependencies.

The **GraphRepresentationGenerator** manages and trains a GNN model. Its most important interfaces include
**the constructor**, which defines the GNN architecture and loads the pre-trained GNN model if it is already on the hard disk,
**the training method**, which initializes the training on the GNN model and
**the get_embedding methods**, which represent the inference interface to the GNN model and return the corresponding embeddings in the dimension defined in the constructor for given user movie node pairs.

**The MovieLensLoader** loads and manages the data sets. The most important tasks include **saving and (re)loading and transforming** the data sets.

**PromptEncoderOnlyClassifier** and **VanillaEncoderOnlyClassifier** each manage a **prompt (model) LLM** and a **vanilla (model) LLM**. An EncoderOnlyClassifier (ClassifierBase) provides interfaces for training and testing an LLM model.
PromptEncoder and VanillaEncoder differ from their DataCollectors. DataCollectors change the behavior of the models during training and testing and allow data points to be created at runtime. With the help of these collators, we **create non-existent edges on the fly**.

In [1]:
from graph_representation_generator import GraphRepresentationGenerator
from dataset_manager import (
    MovieLensManager,
    PROMPT_KGE_DIMENSION,
    ATTENTION_KGE_DIMENSION,
)
from llm_manager import (
    PromptBertClassifier,
    VanillaBertClassifier,
    AttentionBertClassifierBase,
)

In [ ]:
EPOCHS = 20
BATCH_SIZE = 512

We define in advance which **Knowledge Graph Embedding Dimension (KGE_DIMENSION)** the GNN encoder has. We want to determine from which output dimension the GNN encoder can produce embeddings that lead to a significant increase in performance *without exceeding the context length of the LLMs*. In the original tutorial, the KGE_DIMENSION was $64$.

In [ ]:
kg_manager = MovieLensManager()

First we load the MovieLensLoader, which downloads the Movie Lens dataset (https://files.grouplens.org/datasets/movielens/ml-latest-small.zip) and prepares it to be used on GNN and LLM. We also pass the embedding dimensions that we will assume we are training with. First time takes approx. 30 sec.

In [ ]:
kg_manager.data

HeteroData(
  source={ node_id=[610] },
  target={
    node_id=[9742],
    x=[9742, 20],
  },
  (source, edge, target)={ edge_index=[2, 100836] },
  (target, rev_edge, source)={ edge_index=[2, 100836] }
)

Next, we initialize the GNN trainers (possible on Cuda), one for each KGE_DIMENSION.
A GNN trainer manages a model and each model consists of an **encoder and classifier** part.

**The encoder** is a parameterized *Grap Convolutional Network (GCN)* with a *2-layer GNN computation graph* and a single *ReLU* activation function in between.

**The classifier** applies the dot-product between source and destination kges to derive edge-level predictions.

In [ ]:
graph_representation_generator_prompt = GraphRepresentationGenerator(
    kg_manager.data,
    kg_manager.gnn_train_data,
    kg_manager.gnn_val_data,
    kg_manager.gnn_test_data,
    kge_dimension=PROMPT_KGE_DIMENSION,
)
graph_representation_generator_attention = GraphRepresentationGenerator(
    kg_manager.data,
    kg_manager.gnn_train_data,
    kg_manager.gnn_val_data,
    kg_manager.gnn_test_data,
    hidden_channels=ATTENTION_KGE_DIMENSION,
    kge_dimension=ATTENTION_KGE_DIMENSION,
)

loading pretrained model
Device: 'cuda'
loading pretrained model
Device: 'cuda'


We then train and validate the model on the link prediction task.

If the model is already trained, we can skip this part.
Training the models can take up to 5 minutes.

In [ ]:
print("Prompt Training")
#graph_representation_generator_prompt.train_model(kg_manager.gnn_train_data, EPOCHS, BATCH_SIZE)
#graph_representation_generator_prompt.validate_model(kg_manager.gnn_test_data)
print("Attention Training")
#graph_representation_generator_attention.train_model(kg_manager.gnn_train_data, EPOCHS, BATCH_SIZE)
#graph_representation_generator_attention.validate_model(kg_manager.gnn_test_data)


Prompt Training
Attention Training


Next we produce the KGEs for every edge in the dataset. These embeddings can then be used for the LLM on the link-prediction task.

In [ ]:
prompt_embeddings = graph_representation_generator_prompt.get_saved_embeddings("prompt")
attention_embeddings = graph_representation_generator_attention.get_saved_embeddings(
    "attention"
)
save = False
if prompt_embeddings is None or attention_embeddings is None:
    prompt_embeddings = graph_representation_generator_prompt.generate_embeddings(
        kg_manager.llm_df
    )
    attention_embeddings = graph_representation_generator_attention.generate_embeddings(
        kg_manager.llm_df
    )
    save = True

kg_manager.append_prompt_graph_embeddings(prompt_embeddings, save=save)
kg_manager.append_attention_graph_embeddings(attention_embeddings, save=save)


In [ ]:
# kg_manager.add_false_edges(
#     2.0, graph_representation_generator_prompt.get_embedding, graph_representation_generator_attention.get_embedding, splits = ["val", "test"]
# )

Next we initialize the vanilla encoder only classifier. This classifier does only use the NLP part of the prompt (no KGE) for predicting if the given link exists.

In [ ]:
vanilla_bert_classifier = VanillaBertClassifier(
    kg_manager.llm_df, kg_manager.source_df, kg_manager.target_df
)

Next we generate a vanilla llm dataset and tokenize it for training.

In [ ]:
dataset_vanilla = kg_manager.generate_vanilla_dataset(
    vanilla_bert_classifier.tokenize_function
)

Next we train the model on the produced dataset. This can be skipped, if already trained ones.

In [ ]:
#vanilla_bert_classifier.train_model_on_data(dataset_vanilla, epochs=EPOCHS, batch_size = BATCH_SIZE)

Next we initialize the prompt encoder only classifier. This classifier uses the vanilla prompt and the KGEs for its link prediction.

In [ ]:
prompt_bert_classifier = PromptBertClassifier(
    kg_manager,
    graph_representation_generator_prompt.get_embedding,
    model_max_length=512,
)

We also generate a prompt dataset, this time the prompts also include the KGEs.

In [ ]:
dataset_prompt = kg_manager.generate_prompt_embedding_dataset(
    prompt_bert_classifier.tokenize_function,
)

We also train the model. This can be skipped if already done ones.

In [ ]:
#prompt_bert_classifier.train_model_on_data(dataset_prompt, epochs=EPOCHS, batch_size = BATCH_SIZE)

In [ ]:
attention_bert_classifier = AttentionBertClassifierBase(
    kg_manager,
    graph_representation_generator_attention.get_embedding,
)

In [ ]:
dataset_embedding = kg_manager.generate_attention_embedding_dataset(
    attention_bert_classifier.tokenizer.sep_token,
    attention_bert_classifier.tokenizer.pad_token,
    attention_bert_classifier.tokenize_function,
)

In [ ]:
#attention_bert_classifier.train_model_on_data(dataset_embedding, epochs=EPOCHS, batch_size = BATCH_SIZE)

In [ ]:
# kg_manager.add_false_edges(
#     2.0, graph_representation_generator_prompt.get_embedding, graph_representation_generator_attention.get_embedding, splits = ["train"]
# )

In [ ]:
vanilla_bert_classifier = VanillaBertClassifier(
    kg_manager.llm_df,
    kg_manager.source_df,
    kg_manager.target_df,
    false_ratio=-1,
)
prompt_bert_classifier = PromptBertClassifier(
    kg_manager,
    graph_representation_generator_prompt.get_embedding,
    model_max_length=512,
    false_ratio=-1,
)
attention_bert_classifier = AttentionBertClassifierBase(
    kg_manager,
    graph_representation_generator_attention.get_embedding,
    false_ratio=-1,
)


In [ ]:
dataset_vanilla_fixed = kg_manager.generate_vanilla_dataset(
    vanilla_bert_classifier.tokenize_function, suffix="_fixed"
)
dataset_prompt_fixed = kg_manager.generate_prompt_embedding_dataset(
    prompt_bert_classifier.tokenize_function,
    suffix="_fixed",
)
dataset_attention_fixed = kg_manager.generate_attention_embedding_dataset(
    attention_bert_classifier.tokenizer.sep_token,
    attention_bert_classifier.tokenizer.pad_token,
    attention_bert_classifier.tokenize_function,
    suffix="_fixed",
)

In [ ]:
vanilla_bert_classifier.forward_dataset_and_save_outputs(
    dataset_vanilla_fixed, kg_manager.get_vanilla_tokens_as_df, epochs=1, batch_size=BATCH_SIZE
)
prompt_bert_classifier.forward_dataset_and_save_outputs(dataset_prompt_fixed, kg_manager.get_prompt_tokens_as_df, epochs=1)
df = attention_bert_classifier.forward_dataset_and_save_outputs(
    dataset_attention_fixed, kg_manager.get_vanilla_tokens_as_df, epochs=1, batch_size=BATCH_SIZE
)

ValueError: Length of values (3) does not match length of index (90753)